In [1]:
from google.colab import drive

drive.mount('/drive')

Mounted at /drive


In [2]:
#!git clone https://github.com/jwkirchenbauer/lm-watermarking
!pip install -q transformers accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 2.1 MB/s eta 0:00:00


In [3]:
cd '/drive/My Drive/Colab Notebooks/ISO/'

/drive/.shortcut-targets-by-id/1k2JFo63MNzskIgAHpOe33d-BrKTiK7mE/ISO


In [4]:
cd kirchenbauer-lm-watermarking

/drive/.shortcut-targets-by-id/1k2JFo63MNzskIgAHpOe33d-BrKTiK7mE/ISO/kirchenbauer-lm-watermarking


In [5]:
!pip install -q -r new_requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 533.3/533.3 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.1/267.1 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 46.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 20.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 45.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 66.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 77.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 67.7 MB/s

In [6]:
import os
import argparse
import pandas as pd
from argparse import Namespace
from pprint import pprint
from functools import partial

import numpy # for gradio hot reload

import torch
from tqdm import tqdm

from transformers import (AutoTokenizer,
                          AutoModelForSeq2SeqLM,
                          AutoModelForCausalLM,
                          LogitsProcessorList)

from extended_watermark_processor import WatermarkLogitsProcessor, WatermarkDetector

In [7]:
checkpoint = 'facebook/opt-1.3b'

model = AutoModelForCausalLM.from_pretrained(checkpoint, device_map="auto")

device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)
model.eval()

tokenizer = AutoTokenizer.from_pretrained(checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/653 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/685 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/441 [00:00<?, ?B/s]

In [8]:
watermark_detector = WatermarkDetector(vocab=list(tokenizer.get_vocab().values()),
                                        gamma=0.25, # should match original setting
                                        seeding_scheme="algorithm-3", # should match original setting
                                        device=model.device, # must match the original rng device type
                                        tokenizer=tokenizer,
                                        z_threshold=4.0,
                                        normalizers=[],
                                        ignore_repeated_ngrams=True)

In [9]:
cd '../instances'

/drive/.shortcut-targets-by-id/1k2JFo63MNzskIgAHpOe33d-BrKTiK7mE/ISO/instances


In [12]:
def calculate_scores(filename, keyword, custom_filename=None):
    detection_filename = (filename[:-5] + '_detection.json') if custom_filename == None else custom_filename
    df = pd.read_json(filename, lines=True)
    detected = 0
    not_detected = 0
    with open(detection_filename, "r+" if os.path.exists(detection_filename) else "w+") as f:
        with torch.no_grad():
            for i in tqdm(range(len(f.readlines()), len(df["input"]))):
                line = (df["input"][i] if keyword == "continuation" else "") + df[keyword][i]
                # note that if the model is on cuda, then the input is on cuda
                # and thus the watermarking rng is cuda-based.
                # This is a different generator than the cpu-based rng in pytorch!

                score_dict = watermark_detector.detect(line)

                # if decoder only model, then we need to isolate the
                # newly generated tokens as only those are watermarked, the input/prompt is not

                _line = df["input"][i].replace('\n', '\\n').replace('"', '\\"')

                f.write(f'{{"input": "{_line}", "z_score": "{score_dict["z_score"]}", "p_value": "{score_dict["p_value"]}", "prediction": "{1 if score_dict["prediction"] else 0}"}}\n')

                if score_dict["prediction"]:
                    detected += 1
                else:
                    not_detected += 1

    print("\nDetected:", detected)
    print("Not detected:", not_detected)

In [ ]:
calculate_scores('kirchenbauer_no_attack.json', 'continuation')

100%|██████████| 512/512 [00:01<00:00, 426.43it/s]


Detected: 328
Not detected: 184


In [ ]:
calculate_scores('kirchenbauer_paraphrase.json', 'paraphrase')

100%|██████████| 512/512 [00:01<00:00, 420.09it/s]


Detected: 13
Not detected: 499


In [13]:
calculate_scores('kirchenbauer_backtranslated.json', 'translation')

100%|██████████| 512/512 [00:39<00:00, 12.96it/s]


Detected: 61
Not detected: 451


In [14]:
calculate_scores('c4_selection.json', "continuation", "kirchenbauer_c4_detection.json")

100%|██████████| 512/512 [00:53<00:00,  9.55it/s]


Detected: 0
Not detected: 512


In [ ]:
watermark_detector.detect("This is a sample detection")

{'num_tokens_scored': 2,
 'num_green_tokens': 1,
 'green_fraction': 0.5,
 'z_score': 0.8164965809277261,
 'p_value': 0.2071080891212626,
 'z_score_at_T': tensor([-0.5774,  0.8165]),
 'prediction': False}